In [1]:
#####################
# AUTO LOGIN-1 of 1 #
#####################

import login
from login import *
import pandas as pd

0yld9m8WfmP6wDixi9z16BcOprzCeBcz


In [ ]:
#######################
# OPTIONAL RUN-1 of 4 #
#######################

myorder = kite.orders()

myorder

pending_orders = []

for i  in range(len(myorder)):
    if(myorder[i]['status']=='AMO REQ RECEIVED'):
        pending_orders.append(myorder[i])
        kite.cancel_order(myorder[i]['variety'], myorder[i]['order_id'], parent_order_id=None)

pending_orders

In [ ]:
#######################
# OPTIONAL RUN-2 of 4 #
#######################

#kite.positions()['net']

In [ ]:
#######################
# OPTIONAL RUN-3 of 4 #
#######################

#kite.holdings()

In [ ]:
#######################
# OPTIONAL RUN-4 of 4 #
#######################

#kite.margins(segment = 'equity')['available']['live_balance']

In [ ]:
#####################
# NORMAL RUN-1 of 3 #
#####################
#############################################################################################################################
#########################
# Update this part only #
#########################

one_dollar_margin = 1100     # (USD-MIS=984/NRML=1967)(GBP-MIS=1948/NRML=3895)(EUR-MIS=1313/NRML=2625)(JPY-MIS=1333/NRML=2667)
per_order_quant = 25
initial_difference = 0.0000

order_type = 'MIS'           # MIS / NRML
symbol_ip = 'USDINR20OCTFUT' # USDINR20OCTFUT / GBPINR20OCTFUT / EURINR20OCTFUT / JPYINR20OCTFUT
inst_token = 690691          # 690691(USD) / 490755(GBP) / 278019(EUR) / 690435(JPY)
moving_pivot_on = False      # True / False

stoploss_on = False          # True / False

hl_range = 0.23              # (Default=0.00)(USD=0.23)(GBP=?)(EUR=?)(JPY=?)

#############################################################################################################################

import datetime
interval='day'
todaydt=datetime.date.today()
hud_ago=todaydt-datetime.timedelta(days=10)
to_date=datetime.date.isoformat(todaydt)

from_date=datetime.date.isoformat(hud_ago)
data = kite.historical_data(inst_token, from_date, to_date, interval, continuous=False)

import pandas as pd
mydata = pd.DataFrame(data)

last_close = 123.0
last_high = 123.0
last_low = 123.0

from datetime import date

today = date.today()

print('Today Date : '+str(today))


for i in range(len(mydata['date'])):
    if(mydata['date'][i].date()==today-datetime.timedelta(days=1) or mydata['date'][i].date()==today-datetime.timedelta(days=2) or mydata['date'][i].date()==today-datetime.timedelta(days=3)):
        print('Yestreday Date : '+str(mydata['date'][i].date()))
        last_close = mydata['close'][i]
        last_high = mydata['high'][i]
        last_low = mydata['low'][i]
        if(hl_range == 0):
            hl_range = last_high - last_low
        
        
print('last close : '+ str(last_close))
print('last high : '+ str(last_high))
print('last low : '+ str(last_low))
print('HL Range : '+ str(round(hl_range,4)))

mymargin = round(kite.margins(segment = 'equity')['available']['live_balance'],4)
#mymargin = 185777.23

pivot_price = last_close

fullquantity = int(mymargin / one_dollar_margin)

num_of_steps = int(fullquantity / per_order_quant)

difference = round(hl_range / num_of_steps,4)

difference = round(0.0025 * round(difference /0.0025),4)

last_buy_price = round(pivot_price -  difference*num_of_steps,4)
last_sell_price = round(pivot_price +  difference*num_of_steps,4)

print('My Margin : '+str(mymargin))
print('Pivot Price : ' + str(pivot_price))
print('Full Quantity : '+str(fullquantity))
print('Number of Steps : '+str(num_of_steps))
print('Price Difference Between One Trade : ' + str(difference))
print('Quantity Difference Between One Trade : ' + str(per_order_quant))
print('Last Buy Pivot Price : '+str(last_buy_price))
print('Last Sell Pivot Price : '+str(last_sell_price))
print('Stoploss On ? : '+str(stoploss_on))
print('Moving Pivot On ? : '+str(moving_pivot_on))
base=0.0025
max_no_of_order = int(fullquantity / per_order_quant)
total_parts = 2*max_no_of_order

print()
print('pivot_price : '+str(pivot_price))
print()
upper_limit = 0.12
lower_limit = 0.12
def calc_pivots():
    global upper_limit, lower_limit
    for i in range(1,max_no_of_order+1):
        globals()['p'+str(i)+'b'] = pivot_price - (initial_difference + i* difference)
        globals()['p'+str(i)+'b'] = round(base * round(globals()['p'+str(i)+'b'] /base),4)
        print('p'+str(i)+'b : '+str(globals()['p'+str(i)+'b']))

    print()
    for i in range(1,max_no_of_order+1):
        globals()['p'+str(i)+'s'] = pivot_price + (initial_difference + i* difference)
        globals()['p'+str(i)+'s'] = round(base * round(globals()['p'+str(i)+'s'] /base),4)
        print('p'+str(i)+'s : '+str(globals()['p'+str(i)+'s']))

    print()
    for i in range(1,max_no_of_order+1):
        globals()['p'+str(i)+'q'] = per_order_quant*i
        print('p'+str(i)+'q : '+str(globals()['p'+str(i)+'q']))
        
    print()
    upper_limit = round(((globals()['p'+str(max_no_of_order)+'s']) + hl_range*0.236),4)
    lower_limit = round(((globals()['p'+str(max_no_of_order)+'b']) - hl_range*0.236),4)
    print('S Limit : '+str(upper_limit))
    print('B Limit : '+str(lower_limit))

calc_pivots()

In [ ]:
#####################
# NORMAL RUN-2 of 3 #
#####################

myquantity = 0
def getquant():
    global myquantity
    global order_type
    global symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))

getquant()

stopbuy = False
stopsell = False

def placeneworder(quantdiff):
    global stopbuy
    global stopsell
    global order_type
    global symbol_ip
    if(quantdiff>0 and stopbuy == False):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ip,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
                stopsell = False
        except Exception as e:
            stopbuy = False
            print(e)
            getquant()
    if(quantdiff<0 and stopsell == False):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ip,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
                stopbuy = False
        except Exception as e:
            stopsell = True
            print(e)
            getquant()


def ckqnt (orderprice,orderquant):
    global myquantity
    quantdiff = orderquant - myquantity
    placeneworder(quantdiff)
    return myquantity


last_price = pivot_price

def checkpivot(ltps):
    global last_price, upper_limit, lower_limit
    trade_quant = 0
    
    if((ltps > upper_limit or ltps < lower_limit) and stoploss_on ):
        ckqnt(pivot_price,0)
    
    elif ((ltps>=(pivot_price-0.0025)) and (ltps<=(pivot_price+0.0025))):
        ckqnt(pivot_price,0)
        print('in pivot range')
        
    elif(upper_limit>ltps>(globals()['p'+str(max_no_of_order)+'s'])):
        ckqnt(globals()['p'+str(max_no_of_order)+'s'],-globals()['p'+str(max_no_of_order)+'q'])
        
    elif(lower_limit<ltps<(globals()['p'+str(max_no_of_order)+'b'])):
        ckqnt(globals()['p'+str(max_no_of_order)+'b'],globals()['p'+str(max_no_of_order)+'q'])
        
    else:
        for i in range(1,int(total_parts/2)+1):

            if(ltps<pivot_price):
                trade_quant = globals()['p'+str(i)+'q']
            elif(ltps>pivot_price):
                trade_quant = -globals()['p'+str(i)+'q']

            if ((ltps == globals()['p'+str(i)+'b'] or ltps == globals()['p'+str(i)+'s']) and ltps < last_price):
                ckqnt(globals()['p'+str(i)+'b'],trade_quant)
                print('==========' +'p'+str(i)+'b @ ' +str(ltps) + '==============' )
                last_price=ltps

            if ((ltps == globals()['p'+str(i)+'b'] or ltps == globals()['p'+str(i)+'s']) and ltps > last_price):
                ckqnt(globals()['p'+str(i)+'s'],trade_quant)
                print('==========' +'p'+str(i)+'s @ ' +str(ltps) + '==============' )
                last_price=ltps

In [ ]:
#####################
# NORMAL RUN-3 of 3 #
#####################
#################
# START TRADING #
#################

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

def on_ticks(ws, ticks):
    global pivot_price, moving_pivot_on
    checkpivot(ticks[0]['last_price'])

    if(moving_pivot_on):
        pivot_price =round(base * round(ticks[0]['average_price']/base),4) # Moving Trade Average Line : 1
        calc_pivots();                                                     # Moving Trade Average Line : 2
        
    #print(hjhj)
    #print('===========================')
    #print()

        
    
#     print(ticks[0]['last_trade_time'].second)
#     print('==========================')
#     print( 'LTP : ' + str(ticks[0]['last_price']))
    #print('==========================')
    #print()

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

#def on_order_update(ws, data):
#    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
#kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

In [2]:
from datetime import timedelta
import datetime
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()
old_lst=[]
interval='minute'
todaydt=datetime.date.today()
hud_ago=todaydt-timedelta(days=59)
to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)
inst_token = 13734146
for i2 in range(3):
    new_lst = kite.historical_data(inst_token, from_date, to_date, interval,continuous=False)
    old_lst = new_lst + old_lst
    todaydt=todaydt-timedelta(days=60)
    hud_ago=hud_ago-timedelta(days=60)
    to_date=datetime.date.isoformat(todaydt)
    from_date=datetime.date.isoformat(hud_ago)
    print(len(old_lst))

Loading BokehJS ...

7791
7791
7791


In [3]:
mydf = pd.DataFrame(old_lst) 
mydf
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna

# mydf = dropna(mydf)
mydf = add_all_ta_features(
    mydf, open="open", high="high", low="low", close="close", volume="volume")

mydf['hl_range'] = mydf['high'] - mydf['low']
mydf

/home/hemang/.local/lib/python3.8/site-packages/ta/trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/home/hemang/.local/lib/python3.8/site-packages/ta/trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])
/home/hemang/.local/lib/python3.8/site-packages/ta/trend.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dip[i+self._n] = 100 * (self._dip[i]/self._trs[i])
/home/hemang/.local/lib/python3.8/site-packages/ta/trend.py:649: RuntimeWarning: invalid value encountered in double_scalars
  din[i+self._n] = 100 * (self._din[i]/self._trs[i])


,date,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,others_dr,others_dlr,others_cr,hl_range
0,2020-09-21 15:11:00+05:30,41.50,41.50,41.50,41.50,0,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-34.252570,NaN,0.000000,0.00
1,2020-09-21 15:12:00+05:30,41.50,41.50,41.50,41.50,0,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00
2,2020-09-21 15:13:00+05:30,41.50,41.50,41.50,41.50,0,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00
3,2020-09-21 15:14:00+05:30,41.50,41.50,41.50,41.50,0,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00
4,2020-09-21 15:15:00+05:30,41.50,41.50,41.50,41.50,0,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7786,2020-10-26 15:25:00+05:30,16.30,16.45,15.95,16.15,82050,588090.650993,-372525,-0.072136,-1035.369088,...,21.052632,35.350877,-78.947368,0.063529,NaN,-0.308642,-0.920245,-0.924506,-61.084337,0.50
7787,2020-10-26 15:26:00+05:30,16.10,16.30,16.05,16.20,32550,594600.650993,-339975,-0.033122,-654.959218,...,26.315789,29.122807,-73.684211,0.012500,NaN,-1.219512,0.309598,0.309119,-60.963855,0.25
7788,2020-10-26 15:27:00+05:30,16.20,16.30,16.00,16.00,33525,561075.650993,-373500,-0.081461,-1519.250758,...,5.263158,17.543860,-94.736842,-0.048382,NaN,-2.439024,-1.234568,-1.242252,-61.445783,0.30
7789,2020-10-26 15:28:00+05:30,16.10,16.10,15.60,15.85,52950,561075.650993,-426450,-0.042106,-2436.857793,...,19.230769,16.936572,-80.769231,-0.187794,NaN,-2.461538,-0.937500,-0.941922,-61.807229,0.50


In [ ]:
mydf[55:]

In [ ]:
# for i in list(mydf):
#     if(i!='date'):
#         mydf[5:][i].plot.hist(bins=500, alpha=0.8)        

In [ ]:
#  mydf.loc[55:]['hl_range'].plot.hist(bins=250, alpha=0.9)

In [ ]:
# pd.plotting.scatter_matrix(mydf, alpha=0.2)

In [ ]:
# # import matplotlib.pyplot as plt
# # %matplotlib inline 
# ax = plt.figure(num=None, figsize=(100, 100), dpi=500)
# plt.matshow(mydf.corr())
# # plt.show(ax)

In [ ]:
import ta
indicator_rsi = ta.momentum.rsi(close = mydf["close"], n=14, fillna=True)
current_rsi = round(indicator_rsi[-1:].to_numpy()[0],4)
indicator_sma = ta.trend.SMAIndicator(close = mydf["close"], n=14, fillna=True)
current_sma = round(indicator_sma.sma_indicator()[-1:].to_numpy()[0],4)

print('Current RSI : '+str(current_rsi))
print('Current SMA : '+str(current_sma))